In [26]:
from sympy import symbols, solve, Eq
import sympy

def write_line(opened_file, line, tab_count=0):
   opened_file.write("\t" * tab_count + "\\" + line + "\n")

def write_graph_code(file, functions, x_clip, y_clip, samples=[], decimal=False):
   color_order = ["colorEmphasisCyan", "colorEmphasis"]

   with open(file, "w", encoding="utf-8") as f:
      write_line(f, r"begin{figure}[H]")
      write_line(f, "centering", 1)
      write_line(f, r"begin{tikzpicture}", 1)
   
      write_line(f, f"draw[->] ({x_clip[0]}, 0) -- ({x_clip[1]}, 0) node[right] " + "{$x$};", 2)
      write_line(f, f"draw[->] (0, {y_clip[0]}) -- (0, {y_clip[1]})  node[above] " + "{$y$};", 2)

      caption = ""

      for func_id in range(len(functions)):
         x = symbols('x')

         # Get all the possible ending points
         ends = []
         expression = eval(functions[func_id])
         caption += "$" + sympy.latex(expression) + "$"
         if func_id != len(functions) - 1:
            caption += ","

         solutions = solve(Eq(expression, y_clip[0]), x)
         solutions = [sol.evalf() for sol in solutions if sol.is_real]
         ends += solutions

         solutions = solve(Eq(expression, y_clip[1]), x)
         solutions = [sol.evalf() for sol in solutions if sol.is_real]
         ends += solutions

         ends += list(x_clip)

         # Sort and filter the ending points
         ends.sort()
         ends = [x for x in ends if x_clip[0] <= x <= x_clip[1]]
         
         for i in range(len(ends) - 1):
            value = expression.subs(x, ends[i] + 0.05)
            # print(value)

            if value < y_clip[1] and value > y_clip[0]:
               func = functions[func_id].replace("x", r"(\x)").replace("**", "^")
               write_line(f, f"draw[graph thickness, samples=80, color={color_order[func_id]}, domain={ends[i]:.3f}:{ends[i + 1]:.3f}] plot " + r"(\x, {" + func + r"});", 2)
         
         # Graph sample points
         for point in samples:
            if decimal:
               if isinstance(point, tuple):
                  p = float(point[0]) / float(point[1])
               else:
                  p = float(point)

               value = float(sympy.N(expression.subs(x, p)))
               p_latex = sympy.latex(round(p, 3)).replace(".", "{,}")
               value_latex = sympy.latex(round(value, 3)).replace(".", "{,}")
               print(p, value)
               if value < y_clip[1] and value > y_clip[0]:
                  write_line(f, f"filldraw[color= {color_order[func_id]}] ({p}, {value}) circle (\pointSize) node[above] " + r"{$\left(" + f"{p_latex};{value_latex}" + r"\right)$};", 2)
            else:
               if isinstance(point, tuple):
                  p = float(point[0]) / float(point[1])
                  p_rational = sympy.Rational(*point)
               else:
                  p = point
                  p_rational = sympy.Rational(point)
               value = expression.subs(x, p_rational)
               p_latex = sympy.latex(p_rational).replace(".", "{,}")
               value_latex = sympy.latex(value).replace(".", "{,}")
               # print(p_latex, value_latex)
               if value < y_clip[1] and value > y_clip[0]:
                  write_line(f, f"filldraw[color= {color_order[func_id]}] ({p}, {float(value)}) circle (\pointSize) node[above] " + r"{$\left(" + p_latex + ";" + value_latex + r"\right)$};", 2)

      write_line(f, r"end{tikzpicture}", 1)
      write_line(f, r"caption{Đồ thị của " + caption + "}", 1)
      write_line(f, r"end{figure}")

latex_output_file = "output.tex"

write_graph_code(latex_output_file, ["(x**2 - 3*x - 2)/(x**2 + 2*x + 1)"], (-6, 6), (-2.5, 5.5), [-3, -4.5, 0, -0.2, 1, -0.5, 3], True)

-3.0 4.0
-4.5 2.5918367346938775
0.0 -2.0
-0.2 -2.125
1.0 -1.0
-0.5 -1.0
3.0 -0.125


In [ ]:
def write_simple_inequality(file, function1, function2, x_clip, y_clip):
   color_order = ["colorEmphasisCyan", "colorEmphasis"]

   with open(file, "w") as f:
      write_line(f, r"begin{figure}[H]")
      write_line(f, "centering", 1)
      write_line(f, r"begin{tikzpicture}", 1)
   
      # write_line(f, f"clip ({x_clip[0] - 0.5}, {y_clip[0]}) rectangle ({x_clip[1] + 0.5}, {y_clip[1] + 0.5});")
      write_line(f, f"draw[->] ({x_clip[0]}, 0) -- ({x_clip[1]}, 0) node[right] " + "{$x$};", 2)
      write_line(f, f"draw[->] (0, {y_clip[0]}) -- (0, {y_clip[1]})  node[above] " + "{$y$};", 2)

      x = symbols('x')

      # Get the fitting drawing range
      equation = Eq(eval(function1), y_clip[0])
      solutions = solve(equation, x)
      solutions = [sol.evalf() for sol in solutions if sol.is_real]
      left_end1 = solutions[0]

      equation = Eq(eval(function1), y_clip[1])
      solutions = solve(equation, x)
      solutions = [sol.evalf() for sol in solutions if sol.is_real]
      right_end1 = solutions[0]

      func1 = function1.replace("x", r"(\x)").replace("**", "^")
      write_line(f, f"draw[graph thickness, color={color_order[0]}, domain={format(left_end1, ".3f")}:{format(right_end1, ".3f")}] plot " + r"(\x, {" + func1 + r"});", 2)

      # func2 = function2.replace("x", r"(\x)").replace("**", "^")
      # write_line(f, f"draw[graph thickness, color={color_order[0]}, domain={format(left_end, ".3f")}:{format(right_end, ".3f")}] plot " + r"(\x, {" + func2 + r"});", 2)

      write_line(f, r"end{tikzpicture}", 1)
      write_line(f, r"end{figure}")